In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
from functions import read_data

In [3]:
from config import list_cols_with_na

In [4]:
df_train = read_data()

c:\Users\Aaron\projects\kaggle-house_prices\functions.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[list_cols_with_na].fillna("empty", inplace=True)
c:\Users\Aaron\projects\kaggle-house_prices\functions.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[list_cols_with_na].fillna("empty", inplace=True)
c:\Users\Aaron\projects\kaggle-house_prices\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setti

In [ ]:
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_combine_temp = pd.concat([df_train, df_test]).drop(columns=["Id", "SalePrice"])
df_combine_temp.columns

In [ ]:
df_combine_temp["MSZoning"].dtype == "O"